<a href="https://colab.research.google.com/github/pelinbalci/LLM_Notebooks/blob/main/LLM_Zero_One_Few_Shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==1.13.1 --quiet
!pip install torchdata==0.5.1 --quiet
!pip install transformers==4.27.2 --quiet
!pip install datasets==2.11.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 40.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4

## Import Libraries

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

## Load Dataset

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
example_indices = [40, 200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
  print(dash_line)
  print("Ex", i+1)
  print(dash_line)
  print('Input')
  print(dataset['test'][index]['dialogue'])
  print('Human summary')
  print(dataset['test'][index]['summary'])

---------------------------------------------------------------------------------------------------
Ex 1
---------------------------------------------------------------------------------------------------
Input
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
Human summary
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Ex 2
---------------------------------------------------------------------------------------------------
Input
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Pers

# Load Model & Tokenizer

In [ ]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
# How to use tokenizer?
sentence = "What time is it?"
sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_encoded

{'input_ids': tensor([[363,  97,  19,  34,  58,   1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [ ]:
sentence_decoded = tokenizer.decode(sentence_encoded["input_ids"][0],
                                    skip_special_tokens=True)

In [ ]:
sentence_decoded

'What time is it?'

# Zero-Shot Learning

In [ ]:
# select an example
example_index = [200]

# get the dialogue
dialogue = dataset['test'][example_index]['dialogue']

# get the human summary
summary = dataset['test'][example_index]['summary']

# Configurations
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.7)
generation_config = GenerationConfig(max_new_tokens=50)

# Encode input:
inputs_encoded = tokenizer(dialogue, return_tensors='pt')

# Model Output:
model_output = model.generate(inputs_encoded["input_ids"], generation_config=generation_config)[0]

# Decode the output
zero_output = tokenizer.decode(model_output, skip_special_tokens=True)


print("Input: ", dialogue)
print(dash_line)
print( "Human summary: ", summary)
print(dash_line)
print("Model Output: ", zero_output)

Input:  ["#Person1#: Have you considered upgrading your system?\n#Person2#: Yes, but I'm not sure what exactly I would need.\n#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.\n#Person2#: That would be a definite bonus.\n#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.\n#Person2#: How can we do that?\n#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?\n#Person2#: No.\n#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.\n#Person2#: That sounds great. Thanks."]
---------------------------------------------------------------------------------------------------
Human summary:  ["#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."]
---

##One-shot Learning

In [ ]:
def make_prompt(example_indices, example_index_to_sum):
  "write correct prompt from given sentences. Add the latest prompt for predicted sentence"
  prompt = ""

  for index in example_indices:

    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    # prompt includes the dialogue
    prompt += f"""
    Dialogue:

    {dialogue}

    What is going on?
    {summary}

    """

  # get the example dialog
  dialogue = dataset['test'][example_index_to_sum]['dialogue']
  prompt += f"""
    Dialogue:

    {dialogue}

    What is going on?
    """

  return prompt

In [ ]:
example_indices = [40]
example_index_to_sum = 200
one_shot_prompt = make_prompt(example_indices, example_index_to_sum)

print(one_shot_prompt)


    Dialogue:

    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

    What is going on?
    #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

    
    Dialogue:

    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to b

In [ ]:
# Encode input THE ONLY DIFFERENCE IS HERE
inputs_encoded = tokenizer(one_shot_prompt, return_tensors='pt')

# Model Output
model_output = model.generate(inputs_encoded["input_ids"], generation_config=generation_config)[0]

# Decode the output
one_output = tokenizer.decode(model_output, skip_special_tokens=True)

print(dash_line)
print("Ex: ", i+1)
print(dash_line)
print("Input: ", dialogue)
print(dash_line)
print( "Human summary: ", summary)
print(dash_line)
print(" ONE SHOT Output: ", one_output)

---------------------------------------------------------------------------------------------------
Ex:  2
---------------------------------------------------------------------------------------------------
Input:  ["#Person1#: Have you considered upgrading your system?\n#Person2#: Yes, but I'm not sure what exactly I would need.\n#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.\n#Person2#: That would be a definite bonus.\n#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.\n#Person2#: How can we do that?\n#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?\n#Person2#: No.\n#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.\n#Person2#: That sounds great. Thanks."]
----

## Few-shot Learning

In [ ]:
example_indices = [40, 80, 120]
example_index_to_sum = 200
few_shot_prompt = make_prompt(example_indices, example_index_to_sum)

In [ ]:
# Encode input THE ONLY DIFFERENCE IS HERE
inputs_encoded = tokenizer(few_shot_prompt, return_tensors='pt')

# Model Output
model_output = model.generate(inputs_encoded["input_ids"], generation_config=generation_config)[0]

# Decode the output
few_output = tokenizer.decode(model_output, skip_special_tokens=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
print("Input: ", dialogue)
print(dash_line)
print( "Human summary: ", summary)
print(dash_line)
print("Zero-Shot Output: ", zero_output)
print(dash_line)
print("One-Shot Output: ", one_output)
print(dash_line)
print("Few-Shot Output: ", few_output)

Input:  ["#Person1#: Have you considered upgrading your system?\n#Person2#: Yes, but I'm not sure what exactly I would need.\n#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.\n#Person2#: That would be a definite bonus.\n#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.\n#Person2#: How can we do that?\n#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?\n#Person2#: No.\n#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.\n#Person2#: That sounds great. Thanks."]
---------------------------------------------------------------------------------------------------
Human summary:  ["#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system."]
---